In [1]:
import io
import re
import torch
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from huggingface_hub import hf_hub_download

from nltk.tokenize import RegexpTokenizer
from imblearn.over_sampling import RandomOverSampler
from google.colab import files
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback

In [2]:
# 1. Load PhoBERT và Word2Vec
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
phobert_model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=2)

# Load mô hình Word2Vec từ Hugging Face Hub
word2vec_model = KeyedVectors.load_word2vec_format(hf_hub_download(repo_id="Word2vec/nlpl_74", filename="model.bin"), binary=True, unicode_errors="ignore")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.bin:   0%|          | 0.00/1.59G [00:00<?, ?B/s]

In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# 2. Load dữ liệu

train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Report1/train_data.csv')
val_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Report1/val_data.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Report1/test_data.csv')

train_texts = train_df['content'].tolist()
train_labels = train_df['label'].tolist()
val_texts = val_df['content'].tolist()
val_labels = val_df['label'].tolist()
test_texts = test_df['content'].tolist()
test_labels = test_df['label'].tolist()

In [ ]:
# 3. Tokenization với PhoBERT
def tokenize_and_pad(texts, tokenizer, max_length=256):
    encodings = tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=max_length
    )
    return encodings

train_encodings = tokenize_and_pad(train_texts.tolist(), tokenizer, max_length=256)
val_encodings = tokenize_and_pad(val_texts.tolist(), tokenizer, max_length=256)
test_encodings = tokenize_and_pad(test_texts.tolist(), tokenizer, max_length=256)

train_labels = np.array(train_labels).astype(int)
val_labels = np.array(val_labels).astype(int)
test_labels = np.array(test_labels).astype(int)

In [ ]:
# 4. Tạo embedding với Word2Vec
# Trích xuất embedding từ Word2Vec
def get_word2vec_embedding(text, model):
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text.lower())  # Tokenize và chuyển sang chữ thường
    embeddings = []

    for word in words:
        if word in model:
            embeddings.append(model[word])

    if len(embeddings) == 0:
        return np.zeros(model.vector_size)  # Trả về vector không có thông tin nếu không có từ trong model
    return np.mean(embeddings, axis=0)  # Trung bình các embedding của các từ trong câu


In [ ]:
# Kết hợp đặc trưng từ Phobert và Word2Vec
class FakeNewsDataset(Dataset):
    def __init__(self, encodings, labels, texts, word2vec_model, phobert_model):
        self.encodings = encodings
        self.labels = labels
        self.texts = texts
        self.word2vec_model = word2vec_model
        self.phobert_model = phobert_model

    def __getitem__(self, idx):
        # Trích xuất các đặc trưng từ Phobert
        encoding = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        # Lấy input_ids và attention_mask 
        input_ids = encoding['input_ids'].unsqueeze(0)  
        attention_mask = encoding.get('attention_mask', torch.ones_like(input_ids)).unsqueeze(0)

        with torch.no_grad():
            outputs = self.phobert_model(input_ids=input_ids, attention_mask=attention_mask)

        phobert_embedding = outputs.last_hidden_state.mean(dim=1).squeeze()  # Mean pooling

        # Trích xuất đặc trưng từ Word2Vec
        word2vec_embedding = get_word2vec_embedding(self.texts[idx], self.word2vec_model)

        # Kết hợp PhoBERT và Word2Vec embeddings
        combined_embedding = torch.cat((phobert_embedding, torch.tensor(word2vec_embedding, dtype=torch.float32)), dim=0)

        encoding['embedding'] = combined_embedding
        encoding['labels'] = torch.tensor(self.labels[idx]).squeeze()

        return encoding



    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = FakeNewsDataset(train_encodings, train_labels.tolist(), train_texts.tolist(), word2vec_model, phobert_model)
val_dataset = FakeNewsDataset(val_encodings, val_labels.tolist(), val_texts.tolist(), word2vec_model, phobert_model)
test_dataset = FakeNewsDataset(test_encodings, test_labels.tolist(), test_texts.tolist(), word2vec_model, phobert_model)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

<ipython-input-8-62df0ad50142>:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  self.w2v_embeddings = torch.tensor(w2v_embeddings, dtype=torch.float32)


In [ ]:
# Tạo mô hình phân loại 
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=2)

In [10]:
# Hàm huấn luyện cho mô hình kết hợp
def train_combined_model(learning_rate, weight_decay, num_train_epochs):
    training_args = TrainingArguments(
        output_dir='./results_combined',
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=weight_decay,
        logging_dir='./logs_combined',
        logging_steps=10,
        eval_strategy="steps",
        eval_steps=50,
        save_steps=50,
        save_total_limit=3,
        load_best_model_at_end=True,
        learning_rate=learning_rate,
        save_strategy="steps",
    )

    # Khởi tạo Trainer cho mô hình kết hợp
    trainer = Trainer(
        model=phobert_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    trainer.train()

    eval_result = trainer.evaluate()
    return eval_result['eval_loss']

In [11]:
# Tinh chỉnh siêu tham số với Grid Search
param_grid = {
    'learning_rate': [5e-5, 3e-5, 2e-5],
    'weight_decay': [0.01, 0.015, 0.02],
    'num_train_epochs': [3, 4, 5],
}

best_params = None
best_score = float('inf')

# Thực hiện Grid Search
for lr in param_grid['learning_rate']:
    for wd in param_grid['weight_decay']:
        for epochs in param_grid['num_train_epochs']:
            eval_loss = train_combined_model(lr, wd, epochs)
            print(f"Learning rate: {lr}, Weight decay: {wd}, Epochs: {epochs}, Eval loss: {eval_loss}")
            if eval_loss < best_score:
                best_score = eval_loss
                best_params = {'learning_rate': lr, 'weight_decay': wd, 'num_train_epochs': epochs}

print(f"Best params: {best_params}, Best eval loss: {best_score}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

Learning rate: 2e-05, Weight decay: 0.02, Epochs: 3, Eval loss: 0.5787444114685059


Step,Training Loss,Validation Loss
50,0.000100,0.601098
100,0.042600,0.614747
150,0.000100,0.597877
200,0.110000,0.628093
250,0.000100,0.543819
300,0.000100,0.723405
350,0.188300,0.399182
400,0.000300,0.536056
450,0.115500,0.519132
500,0.141800,0.533668


Learning rate: 2e-05, Weight decay: 0.02, Epochs: 4, Eval loss: 0.3991822302341461


Step,Training Loss,Validation Loss
50,0.000300,0.427671
100,0.055100,0.589275
150,0.000100,0.491046
200,0.000100,0.551047


Learning rate: 2e-05, Weight decay: 0.02, Epochs: 5, Eval loss: 0.42767131328582764
Best params: {'learning_rate': 3e-05, 'weight_decay': 0.015, 'num_train_epochs': 3}, Best eval loss: 0.2419527918100357


In [13]:
# Khởi tạo Trainer với Dừng sớm (Early Stopping)
training_args_with_early_stop = TrainingArguments(
    output_dir='./results_with_early_stop',
    num_train_epochs=best_params['num_train_epochs'],
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=best_params['weight_decay'],
    logging_dir='./logs_with_early_stop',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    save_total_limit=3,
    load_best_model_at_end=True,
    learning_rate=best_params['learning_rate'],
)

trainer_with_early_stop = Trainer(
    model=phobert_model,
    args=training_args_with_early_stop,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [26]:
trainer_with_early_stop.train()

Step,Training Loss,Validation Loss
50,0.104200,0.416946
100,0.131300,0.484746
150,0.000400,0.401399
200,0.086300,0.375797
250,0.148600,0.477886
300,0.064300,0.576847
350,0.005600,0.488487
400,0.264600,0.413812


TrainOutput(global_step=400, training_loss=0.05901278503995854, metrics={'train_runtime': 667.9228, 'train_samples_per_second': 8.988, 'train_steps_per_second': 1.127, 'total_flos': 420056799882240.0, 'train_loss': 0.05901278503995854, 'epoch': 1.593625498007968})

In [27]:
# Dự đoán nhãn cho tập kiểm tra
predictions2 = trainer_with_early_stop.predict(val_dataset)

# Lấy nhãn dự đoán từ logits
pred_labels2 = np.argmax(predictions2.predictions, axis=1)

In [29]:
# Tính các chỉ số
accuracy2 = accuracy_score(val_labels, pred_labels2)
precision2 = precision_score(val_labels, pred_labels2, pos_label=0)
recall2 = recall_score(val_labels, pred_labels2, pos_label=0)
f12 = f1_score(val_labels, pred_labels2, pos_label=0)
auc2 = roc_auc_score(val_labels, predictions2.predictions[:, 1])

print(f"Accuracy: {accuracy2:.6f}")
print(f"Precision: {precision2:.6f}")
print(f"Recall: {recall2:.6f}")
print(f"F1 Score: {f12:.6f}")
print(f'AUC: {auc2:.6f}')

Accuracy: 0.950100
Precision: 0.925926
Recall: 0.980392
F1 Score: 0.952381
AUC: 0.985477


In [30]:
# Tính các chỉ số
accuracy2 = accuracy_score(val_labels, pred_labels2)
precision2 = precision_score(val_labels, pred_labels2, pos_label=1)
recall2 = recall_score(val_labels, pred_labels2, pos_label=1)
f12 = f1_score(val_labels, pred_labels2, pos_label=1)
auc2 = roc_auc_score(val_labels, predictions2.predictions[:, 1])

print(f"Accuracy: {accuracy2:.6f}")
print(f"Precision: {precision2:.6f}")
print(f"Recall: {recall2:.6f}")
print(f"F1 Score: {f12:.6f}")
print(f'AUC: {auc2:.6f}')

Accuracy: 0.950100
Precision: 0.978355
Recall: 0.918699
F1 Score: 0.947589
AUC: 0.985477
